# 製品プライサーは継続しました

その説明から、何かがどれだけの費用がかかるかを推定できるモデル。

##ベースラインモデル

今日、私たちは最も単純なモデルに取り組み、打ち負かす出発点として機能します。

In [ ]:
# 輸入

import os
import math
import json
import random
from dotenv import load_dotenv
from huggingface_hub import login
import matplotlib.pyplot as plt
import numpy as np
import pickle
from collections import Counter

In [ ]:
# 従来の機械学習のためのより多くの輸入

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

## NLPインポート

次のセルでは、NLP関連の機械学習のための輸入品が増えています。  
Gensimのインポートが「scipy.linalg」から「name 'triu」をインポートできないようなエラーを提供する場合は、別のセルで実行してください。  
`！pipインストール" scipy <1.13 "`  
StackoverFlow [here](https://stackoverflow.com/questions/78279136/importerror-cannot-import-name-triu-from-scipy-linalg-when-importing-gens)で説明されているとおり。  
これを並べ替えてくれたArnaldo GとArd Vに感謝します。

In [ ]:
# NLP関連の輸入

from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

In [ ]:
# 最後に、より高度な機械学習のためのより多くの輸入

from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# 定数 - 色のstdoutへの印刷に使用されます

GREEN = "\033[92m"
YELLOW = "\033[93m"
RED = "\033[91m"
RESET = "\033[0m"
COLOR_MAP = {"red":RED, "orange": YELLOW, "green": GREEN}

In [ ]:
# 環境

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [ ]:
# Huggingfaceにログインします

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

In [ ]:
# ログイン後にもう1つインポートします

from items import Item

In [ ]:
%matplotlib inline

# PKLファイルのロード

すべてのデータを再度キュレーションすることを避けましょう！ピクルスファイルにロードします

2日目にまだこれらを作成していない場合は、Googleドライブからダウンロードすることもできます（ここにスライドもあります）：  
https://drive.google.com/drive/folders/1jwnorprhdnf_pu0ge5yytfklyrkc3cov？usp = sharing

しかし、ファイルは非常に大きいことに注意してください - コーヒーを手に入れる必要があるかもしれません！

In [ ]:
with open('train.pkl', 'rb') as file:
    train = pickle.load(file)

with open('test.pkl', 'rb') as file:
    test = pickle.load(file)

In [ ]:
# トレーニングプロンプトを思い出させてください

print(train[0].prompt)

In [ ]:
# テストプロンプトを思い出させます

print(train[0].price)

## たくさん使用する強力なスクリプトを発表します！

テストセットから250アイテムに対してモデルを評価するかなり心地よいテストハーネス

そして、結果を視覚的に満足のいく方法で見せてください。

このフォームの関数を書きます。

```
def my_prediction_function(item):
    # my code here
    return my_estimate
```

そして、あなたは電話します：

`tester.test（my_prediction_function）`

モデルを評価します。

In [ ]:
class Tester:

    def __init__(self, predictor, title=None, data=test, size=250):
        self.predictor = predictor
        self.data = data
        self.title = title or predictor.__name__.replace("_", " ").title()
        self.size = size
        self.guesses = []
        self.truths = []
        self.errors = []
        self.sles = []
        self.colors = []

    def color_for(self, error, truth):
        if error<40 or error/truth < 0.2:
            return "green"
        elif error<80 or error/truth < 0.4:
            return "orange"
        else:
            return "red"
    
    def run_datapoint(self, i):
        datapoint = self.data[i]
        guess = self.predictor(datapoint)
        truth = datapoint.price
        error = abs(guess - truth)
        log_error = math.log(truth+1) - math.log(guess+1)
        sle = log_error ** 2
        color = self.color_for(error, truth)
        title = datapoint.title if len(datapoint.title) <= 40 else datapoint.title[:40]+"..."
        self.guesses.append(guess)
        self.truths.append(truth)
        self.errors.append(error)
        self.sles.append(sle)
        self.colors.append(color)
        print(f"{COLOR_MAP[color]}{i+1}: Guess: ${guess:,.2f} Truth: ${truth:,.2f} Error: ${error:,.2f} SLE: {sle:,.2f} Item: {title}{RESET}")

    def chart(self, title):
        max_error = max(self.errors)
        plt.figure(figsize=(12, 8))
        max_val = max(max(self.truths), max(self.guesses))
        plt.plot([0, max_val], [0, max_val], color='deepskyblue', lw=2, alpha=0.6)
        plt.scatter(self.truths, self.guesses, s=3, c=self.colors)
        plt.xlabel('Ground Truth')
        plt.ylabel('Model Estimate')
        plt.xlim(0, max_val)
        plt.ylim(0, max_val)
        plt.title(title)
        plt.show()

    def report(self):
        average_error = sum(self.errors) / self.size
        rmsle = math.sqrt(sum(self.sles) / self.size)
        hits = sum(1 for color in self.colors if color=="green")
        title = f"{self.title} Error=${average_error:,.2f} RMSLE={rmsle:,.2f} Hits={hits/self.size*100:.1f}%"
        self.chart(title)

    def run(self):
        self.error = 0
        for i in range(self.size):
            self.run_datapoint(i)
        self.report()

    @classmethod
    def test(cls, function):
        cls(function).run()

# 今、基本的な何かのために

想像できる最もシンプルなモデルは何ですか？

乱数ジェネレーターから始めましょう！

In [ ]:
def random_pricer(item):
    return random.randrange(1,1000)

In [ ]:
# ランダムシードを設定します

random.seed(42)

# TestRunnerを実行します
Tester.test(random_pricer)

In [ ]:
# 楽しかったです！
# 私たちはもっとうまくやることができます - ここに別のかなり些細なモデルがあります

training_prices = [item.price for item in train]
training_average = sum(training_prices) / len(training_prices)

def constant_pricer(item):
    return training_average

In [ ]:
# 一定の予測因子を実行します
Tester.test(constant_pricer)


In [ ]:
train[0].details

In [ ]:
# アイテムに新しい「機能」フィールドを作成し、詳細dictから解析されたJSONを入力します

for item in train:
    item.features = json.loads(item.details)
for item in test:
    item.features = json.loads(item.details)

# 1つを見てください

In [ ]:
train[0].features.keys()

In [ ]:
# トレーニングセットの20の最も一般的な機能を見てください

feature_count = Counter()
for item in train:
    for f in item.features.keys():
        feature_count[f]+=1

feature_count.most_common(40)

In [ ]:
# 今、アイテムの重量を摘むためのいくつかのJankyコード
# これについてあまり心配しないでください：スポイラーアラート、トレーニングではあまり役に立たないでしょう！

def get_weight(item):
    weight_str = item.features.get('Item Weight')
    if weight_str:
        parts = weight_str.split(' ')
        amount = float(parts[0])
        unit = parts[1].lower()
        if unit=="pounds":
            return amount
        elif unit=="ounces":
            return amount / 16
        elif unit=="grams":
            return amount / 453.592
        elif unit=="milligrams":
            return amount / 453592
        elif unit=="kilograms":
            return amount / 0.453592
        elif unit=="hundredths" and parts[2].lower()=="pounds":
            return amount / 100
        else:
            print(weight_str)
    return None

In [ ]:
weights = [get_weight(t) for t in train]
weights = [w for w in weights if w]

In [ ]:
average_weight = sum(weights)/len(weights)
average_weight

In [ ]:
def get_weight_with_default(item):
    weight = get_weight(item)
    return weight or average_weight

In [ ]:
def get_rank(item):
    rank_dict = item.features.get("Best Sellers Rank")
    if rank_dict:
        ranks = rank_dict.values()
        return sum(ranks)/len(ranks)
    return None

In [ ]:
ranks = [get_rank(t) for t in train]
ranks = [r for r in ranks if r]
average_rank = sum(ranks)/len(ranks)
average_rank

In [ ]:
def get_rank_with_default(item):
    rank = get_rank(item)
    return rank or average_rank

In [ ]:
def get_text_length(item):
    return len(item.test_prompt())

In [ ]:
# ブランドを調査します

brands = Counter()
for t in train:
    brand = t.features.get("Brand")
    if brand:
        brands[brand]+=1

# 最も一般的な40のブランドを見てください

brands.most_common(40)

In [ ]:
TOP_ELECTRONICS_BRANDS = ["hp", "dell", "lenovo", "samsung", "asus", "sony", "canon", "apple", "intel"]
def is_top_electronics_brand(item):
    brand = item.features.get("Brand")
    return brand and brand.lower() in TOP_ELECTRONICS_BRANDS

In [ ]:
def get_features(item):
    return {
        "weight": get_weight_with_default(item),
        "rank": get_rank_with_default(item),
        "text_length": get_text_length(item),
        "is_top_electronics_brand": 1 if is_top_electronics_brand(item) else 0
    }

In [ ]:
# トレーニングアイテムの機能を見てください
get_features(train[0])

In [ ]:
# 機能をパンダのデータフレームに変換するユーティリティ機能

def list_to_dataframe(items):
    features = [get_features(item) for item in items]
    df = pd.DataFrame(features)
    df['price'] = [item.price for item in items]
    return df

train_df = list_to_dataframe(train)
test_df = list_to_dataframe(test[:250])

In [ ]:
# 従来の線形回帰！

np.random.seed(42)

# 個別の機能とターゲット
feature_columns = ['weight', 'rank', 'text_length', 'is_top_electronics_brand']

X_train = train_df[feature_columns]
y_train = train_df['price']
X_test = test_df[feature_columns]
y_test = test_df['price']

# 線形回帰を訓練します
model = LinearRegression()
model.fit(X_train, y_train)

for feature, coef in zip(feature_columns, model.coef_):
    print(f"{feature}: {coef}")
print(f"Intercept: {model.intercept_}")

# テストセットを予測して評価します
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {r2}")

In [ ]:
# 新しいアイテムの価格を予測する機能

def linear_regression_pricer(item):
    features = get_features(item)
    features_df = pd.DataFrame([features])
    return model.predict(features_df)[0]

In [ ]:
# それをテストします

Tester.test(linear_regression_pricer)

In [ ]:
# 次のいくつかのモデルについては、文書と価格を準備します
# ドキュメントにテストプロンプトを使用していることに注意してください。そうでなければ、答えを明らかにします!!

prices = np.array([float(item.price) for item in train])
documents = [item.test_prompt() for item in train]

In [ ]:
# 単語モデルの袋にはcountvectorizerを使用します

np.random.seed(42)
vectorizer = CountVectorizer(max_features=1000, stop_words='english')
X = vectorizer.fit_transform(documents)
regressor = LinearRegression()
regressor.fit(X, prices)

In [ ]:
def bow_lr_pricer(item):
    x = vectorizer.transform([item.test_prompt()])
    return max(regressor.predict(x)[0], 0)

In [ ]:
# それをテストします

Tester.test(bow_lr_pricer)

In [ ]:
# Gensim NLPライブラリに実装されている驚くべきWord2Vecモデル

np.random.seed(42)

# ドキュメントを前処理します
processed_docs = [simple_preprocess(doc) for doc in documents]

# word2vecモデルをトレーニングします
w2v_model = Word2Vec(sentences=processed_docs, vector_size=400, window=5, min_count=1, workers=8)

In [ ]:
# ドキュメント全体でベクトルを平均化するこのステップは、私たちのアプローチの弱点です

def document_vector(doc):
    doc_words = simple_preprocess(doc)
    word_vectors = [w2v_model.wv[word] for word in doc_words if word in w2v_model.wv]
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(w2v_model.vector_size)

# 機能マトリックスを作成します
X_w2v = np.array([document_vector(doc) for doc in documents])

In [ ]:
# word2vecで線形回帰を実行します

word2vec_lr_regressor = LinearRegression()
word2vec_lr_regressor.fit(X_w2v, prices)

In [ ]:
def word2vec_lr_pricer(item):
    doc = item.test_prompt()
    doc_vector = document_vector(doc)
    return max(0, word2vec_lr_regressor.predict([doc_vector])[0])

In [ ]:
Tester.test(word2vec_lr_pricer)

In [ ]:
# ベクターマシンをサポートします

np.random.seed(42)
svr_regressor = LinearSVR()

svr_regressor.fit(X_w2v, prices)

In [ ]:
def svr_pricer(item):
    np.random.seed(42)
    doc = item.test_prompt()
    doc_vector = document_vector(doc)
    return max(float(svr_regressor.predict([doc_vector])[0]),0)

In [ ]:
Tester.test(svr_pricer)

In [ ]:
# 強力なランダムフォレスト回帰

rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=8)
rf_model.fit(X_w2v, prices)

In [ ]:
def random_forest_pricer(item):
    doc = item.test_prompt()
    doc_vector = document_vector(doc)
    return max(0, rf_model.predict([doc_vector])[0])

In [ ]:
Tester.test(random_forest_pricer)